In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
# from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, Activation, Rescaling
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential as SequentialModel
import keras_tuner as kt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import numpy as np
import matplotlib.pyplot as plt
import itertools

# tf.random.set_seed(5263)
tf.__version__

In [ ]:
# Set up datasets before passing to the model


# TODO: Image shape should be taken from the dataset, not hardcoded
image_shape = (500, 500, 3)

# Normalise image values to 0-1 from 0-255
img_normaliser = layers.Rescaling(1./255)

In [ ]:
def wamai_model_builder(hp):
    
    # Tune whether to pool image to half or quarter size before passing it to the high-filter convolutional layer - revisit if necessary
    # hp_pool_size = hp.Choice('learning_rate', values=[(2, 2), (4, 4)])
    
    # Tuning filter count for convolutional layers
    hp_conv1_filter_count = hp.Int('conv_1_filters', min_value=16, max_value=64, step=8)
    hp_conv2_filter_count = hp.Int('conv_2_filters', min_value=32, max_value=128, step=16)
    hp_conv3_filter_count = hp.Int('conv_3_filters', min_value=64, max_value=256, step=16)
    
    # Tuning amount of neurons in hidden layers after flattening
    hp_hidden1_units = hp.Int('first_hidden_units', min_value=256, max_value=2048, step=64)
    hp_hidden2_units = hp.Int('second_hidden_units', min_value=128, max_value=1024, step=64)
    
    wamai_model = SequentialModel([

    layers.Conv2D(filters = hp_conv1_filter_count, kernel_size = (3,3), strides = 2, input_shape = image_shape),
    layers.Activation('relu'),
    layers.BatchNormalization(),
    
    layers.Conv2D(filters = hp_conv2_filter_count, kernel_size = (3,3), strides = 2),
    layers.Activation('relu'),
    layers.BatchNormalization(),
    
    layers.MaxPool2D(pool_size = (2, 2)),
    layers.Conv2D(filters = hp_conv3_filter_count, kernel_size = (3,3), strides = 2),
    layers.Activation('relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    
    layers.Dense(units = hp_hidden1_units),
    layers.Activation('relu'),
    
    layers.Dense(units = hp_hidden2_units),
    layers.Activation('relu'),
    
    layers.Dense(units = 8),
    layers.Activation('sigmoid')

    ])
    
    wamai_model.compile(
                      optimizer='adam', loss='binary_crossentropy',
                      metrics=['accuracy'], loss_weights=None,
                      weighted_metrics=None, run_eagerly=None,
                      steps_per_execution=None
    )
    
    return wamai_model

# Tune the model probabilistically, using accuracy as the metric to optimise the model
tuner = kt.tuners.BayesianOptimization(wamai_model_builder, objective="accuracy", max_trials=20, project_name="income_predictor_bayesian")
# Search for the best hyperparameters
tuner.search(features_train, target_train, validation_data=(features_test, target_test))
best_hps = tuner.get_best_hyperparameters()[0]


In [ ]:
# wamai_model = SequentialModel([
# 
#     layers.Conv2D(filters = 32, kernel_size = (3,3), strides = 2, input_shape = image_shape),
#     layers.Activation('relu'),
#     layers.BatchNormalization(),
#     
#     layers.Conv2D(filters = 64, kernel_size = (3,3), strides = 2),
#     layers.Activation('relu'),
#     layers.BatchNormalization(),
#     
#     layers.Conv2D(filters = 128, kernel_size = (3,3), strides = 2),
#     layers.Activation('relu'),
#     layers.BatchNormalization(),
#     
#     layers.MaxPool2D(pool_size = (2, 2)),
#     layers.Conv2D(filters = 32, kernel_size = (3,3)),
#     layers.Activation('relu'),
#     layers.BatchNormalization(),
#     
#     layers.MaxPool2D(pool_size = (2, 2)),
#     layers.Flatten(),
#     
#     layers.Dense(units = 512),
#     layers.Activation('relu'),
#     
#     layers.Dense(units = 512),
#     layers.Activation('relu'),
#     
#     layers.Dense(units = 1),
#     layers.Activation('sigmoid')
# 
# ])

In [ ]:
# Build and train the model - note that training time will be long if on CPU.
wamai_model.compile(
                      optimizer='adam', loss='categorical_crossentropy',
                      metrics=['accuracy'], loss_weights=None,
                      weighted_metrics=None, run_eagerly=None,
                      steps_per_execution=None
)

print(wamai_model.summary())

# history = classification_model.fit(train_dataset, validation_data=test_dataset, epochs=20)